In [1]:
# for Exited, 1 is not-exited, -1 is exited
# for gender, male is 0 and female is 1


#Consider creating a combined column for exited and active member
    #-2 if exited and active, -1 if exited and inactive, 1 if stayed and inactive, 2 if stayed and active

#clean outlier data from monthly salary (scale or remove?)

#need to do either undersampling or oversampling, about 8000 users stayed while only 2000 left

#standardscaler or create new features from the features (salary, balance, age, credit score, tenure)

# separate data by region? looks like most data is from france. Much less likely for french customers to leave.
# i.e. the leave data for all three countries is similar but the stay data is much higher in france.

#MY ASSUMPTIONS:
#The data was collected at two points in a year. First point gave balance, salary, etc., then later that year they checked if that person had left or stayed
#Balance may still exist if a person left the bank because it represents the balance they had before leaving
#User is considered an active user if they are still using the bank services in some way. its possible for an active member to leave. its possible for an inactive member to stay.

#is profitable idea: check a 1 for yes and -1 for no, based on iff the user is both an active member AND has a positive balance

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [3]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners.csv"))
starter_df.sample(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2345,2346,15641250,Calabresi,794,Spain,Male,38,9,179581.31,1,1,0,23596.24,0
7633,7634,15727811,Ts'ui,661,Germany,Female,47,0,109493.62,1,0,0,188324.01,1
5167,5168,15587174,Kerr,726,France,Male,29,7,0.00,2,1,1,91844.14,1
2563,2564,15645463,Udinese,843,France,Female,27,5,0.00,2,1,1,67494.23,0
6812,6813,15605059,Mackie,576,Germany,Male,63,3,148843.56,1,1,0,69414.13,1
4429,4430,15686463,Fu,626,France,Male,38,7,141074.59,1,1,0,52795.56,1
4581,4582,15746127,Hort,572,France,Female,47,2,0.00,2,1,0,36099.70,0
3910,3911,15698716,Baker,620,France,Female,70,3,87926.24,2,1,0,33350.26,1
15,16,15643966,Goforth,616,Germany,Male,45,3,143129.41,2,0,1,64327.26,0
725,726,15672692,Yin,787,France,Female,42,10,145988.65,2,1,1,79510.37,0


In [4]:
#Convert the values of the Gender column from "M" or "F" to "0" or "1"
starter_df = starter_df.replace({'Gender' : {'Male': 0, 'Female': 1}})
#Make HasCrCard a -1 if they don't, so it reflects thats a negative
starter_df = starter_df.replace({'HasCrCard' : {1: -1, 0 : 1}})
#Make IsActiveMember a -1 if they arent, so it reflects thats a negative
starter_df = starter_df.replace({'Exited' : {0: -1}})
#Make IsActiveMember a -1 if they arent, so it reflects thats a negative
starter_df = starter_df.replace({'IsActiveMember' : {0: -1}})
#Drop the useless columns
starter_df = starter_df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
#Drop Nan's
starter_df.dropna(inplace=True)

In [5]:
starter_df.sample(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
338,805,Spain,1,39,5,165272.13,1,-1,-1,14109.85,1
1188,528,France,0,30,2,128262.72,2,-1,-1,50771.16,-1
8142,615,France,0,41,4,130385.82,1,1,1,130661.95,-1
5428,507,France,1,35,2,0.00,2,-1,-1,97633.93,-1
7088,556,France,1,54,4,150005.38,1,-1,-1,157015.50,1
7567,629,France,0,44,5,0.00,4,1,-1,117572.59,1
2229,757,Spain,0,44,5,140856.16,2,-1,-1,158735.10,-1
862,725,France,0,41,7,113980.21,1,-1,1,116704.25,-1
6844,587,France,0,33,8,148163.57,1,1,-1,122925.40,-1
8289,682,France,0,40,4,0.00,1,1,1,105352.55,-1


In [6]:
#Not sure if I should use this, since im doing feature creation next on these two unscaled variables
scaler = StandardScaler()
starter_df[['ScaledBalance', 'ScaledEstimatedSalary']] = scaler.fit_transform(starter_df[['Balance', 'EstimatedSalary']])

In [7]:
#use feature creation to make a ratio between age and tenure
starter_df['BalanceSalaryRatio'] = (starter_df['Balance'] / starter_df['EstimatedSalary']).round(2)
starter_df.drop(columns=['Balance', 'EstimatedSalary'], inplace=True)

In [8]:
# Create a OneHotEncoder instance
#enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
#encoded_data = enc.fit_transform(starter_df[['Geography']])

In [9]:
#encoded_df_rough = pd.DataFrame(
#encoded_data,
#columns = enc.get_feature_names(['Geography']))

In [10]:
#use feature creation to make a ratio between age and tenure
starter_df['TenureByAge'] = (starter_df['Tenure'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure', 'Age'], inplace=True)

In [11]:
#Remove geography. We are just doing France.
starter_df = starter_df[starter_df.Geography != 'Spain']
starter_df = starter_df[starter_df.Geography != 'Germany']
starter_df.drop(columns=['Geography'], inplace=True)

In [12]:
"""
encoded_df = pd.concat(
    [
        encoded_df_rough,
        starter_df
    ],
    axis=1
)
encoded_df.sample(10)
"""

'\nencoded_df = pd.concat(\n    [\n        encoded_df_rough,\n        starter_df\n    ],\n    axis=1\n)\nencoded_df.sample(10)\n'

In [13]:
X = starter_df.drop(columns= 'Exited')

In [14]:
y = starter_df['Exited']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
starter_df['Exited'].value_counts()

-1    4204
 1     810
Name: Exited, dtype: int64

In [17]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [18]:
'''
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
'''

In [19]:
#Choose a model
#clf = SVC()
#clf = LogisticRegression()
clf = RandomForestClassifier(n_estimators= 2000)

In [20]:
#Fit the model on the training data
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000)

In [21]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [22]:
accuracy = accuracy_score(test_predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 81.10%


In [23]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": test_predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(5)

,Testing Data Predictions,Testing Data Actual Targets
9972,1,-1
9977,-1,-1
9983,-1,-1
9985,1,-1
9996,-1,-1


In [24]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[935 111]
 [126  82]]


In [25]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

          -1       0.88      0.89      0.89      1046
           1       0.42      0.39      0.41       208

    accuracy                           0.81      1254
   macro avg       0.65      0.64      0.65      1254
weighted avg       0.81      0.81      0.81      1254

